<a href="https://colab.research.google.com/github/nabilah-afrin/recommendation_system_rokomri_books/blob/secondary/notebooks/preprocessing_bn_books_title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Rokomari Recommendation Dataset

# %cd /content/drive/MyDrive/Dataset/rokomari_books

/content/drive/MyDrive/Rokomari Recommendation Dataset


In [3]:
!ls

category_counts.csv	df_bn.csv			    rokomari_bn_books_author_corrected.csv
category_counts.gsheet	preprocessing_bn_books_title.ipynb  rokomari_books_categories_corrected.csv


In [3]:
# !pip install googletrans==4.0.0-rc1
!pip install googletrans==4.0.0rc1 --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.4 MB/s eta 0:00:00


In [4]:
!pip install langdetect --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
!pip install deep_translator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00


# Import

In [6]:
import re
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import langdetect as ld
import googletrans as gt
import deep_translator as dt
from deep_translator import GoogleTranslator

# Useful fns

In [7]:
import re
import time
# from easynmt import EasyNMT
# model = EasyNMT('m2m_100_1.2B')
# print(model.translate('Bank Job Entrance Exam Preparation', target_lang='bn'))

def detect_language(row):
    try:
        lang = ld.detect(str(row))
    except:
        return 'unknown'
    return lang

def keep_only_bangla(row):
    """
    The unicode range between \u0980 - \u09FF defines the Bangla characters
    and digits in the Unicode character set
    """
    # return re.sub(r'[^\u0980-\u09FF ]+', '', str(row)).strip()
    return re.sub(r'[^\u0980-\u09FF\u0041-\u005A\u0061-\u007A :?]+', '', str(row)).strip()

def translate_author_names(row):
    text = str(row)
    # time.sleep(8)
    translator = gt.Translator()
    translation = translator.translate(text=text, dest='bn')

    return translation.text

def translate_to_bangla(row):
    return GoogleTranslator(source='auto', target='bn').translate(text=str(row))



# 1. Load Dataframe

In [8]:
df_bn = pd.read_csv("rokomari_books_categories_corrected.csv")

In [10]:
df_bn

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed
0,লু স্যুন,পশ্চিমবঙ্গের বই: সমকালীন গল্প,9788194097303,No Ratings,No Reviews,320,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,নির্বাচিত গল্প সংকলন,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সমকালীন গল্প"
1,আর্কাদি গাইদার,পশ্চিমবঙ্গের বই: শিশু-কিশোর উপন্যাস,9788193860939,No Ratings,No Reviews,180,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,তিমুর ও তার দলবল,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, শিশু-কিশোর উপন্যাস"
2,মতি নন্দী,পশ্চিমবঙ্গের বই: সমকালীন উপন্যাস,No ISBN,No Ratings,No Reviews,250,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,নায়কের প্রবেশ ও প্রস্থান,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সমকালীন উপন্যাস"
3,উৎপলকুমার বসু,পশ্চিমবঙ্গের বই: বাংলা কবিতা,No ISBN,No Ratings,No Reviews,300,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,হাঁস চলার পথ,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, বাংলা কবিতা"
4,দিলরুবা হাসান,বয়স যখন ৪-৮: শিক্ষামূলক,No ISBN,No Ratings,No Reviews,129,No image,বৃত্তকলা একাডেমি,0.0,No summary,অনুভূতির ছোঁয়া,bn,No,bn,bn,"বয়স যখন ৪-৮, শিক্ষামূলক"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98249,আ শ ম বাবর আলী,বয়স যখন ৪-৮: বাংলা ছড়া,9789849338253,No Ratings,No Reviews,129,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থ প্রকাশ,0.0,No summary,আবোল তাবোল ছড়া,bn,No,bn,bn,"বয়স যখন ৪-৮, বাংলা ছড়া"
98250,বিলাল হোসাইন নূরী,বয়স যখন ৮-১২: গল্প,9789849418405,No Ratings,No Reviews,144,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থকুঞ্জ প্রকাশন,0.0,No summary,কুসুম ফোটার বেলা,bn,No,bn,bn,"বয়স যখন ৮-১২, গল্প"
98251,সতীনাথ ভাদুড়ী,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,No ISBN,No Ratings,No Reviews,360,No image,গ্রন্থালয় (ভারত),0.0,No summary,সতীনাথ ভাদুড়ীঃ শতবার্ষিকী রচনা সংকলন ২য় খণ্ড,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, রচনাসমগ্র, সংকলন"
98252,চিত্তরঞ্জন ঘোষাল,হিন্দু ধর্মীয় বই,No ISBN,No Ratings,No Reviews,720,No image,গ্রন্থিক (ভারত),0.0,No summary,গীতা সংগ্রহ: শ্রীমদভগবদ্গীতা-সহ ৩৫টি গীতা একখণ...,bn,No,bn,bn,হিন্দু ধর্মীয় বই


# 2. Look for title's language types


In [ ]:
# df_bn['title'].apply(detect_language).value_counts()


In [9]:
df_bn['title_lang'] = df_bn['title'].apply(detect_language)

In [10]:
df_bn['title_lang'].value_counts()


,count
title_lang,
bn,98204
ar,48
fa,2


In [17]:
df_bn.loc[df_bn['title_lang'] == 'en']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
9954,এসেনশিয়াল পাবলিকেশন সম্পাদক,টোফেল,No ISBN,No Ratings,No Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,0.0,CONTENTS,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,Yes,bn,bn,টোফেল,en,Cliffs Toefl Grand Review বাংলা ভার্সন


In [11]:
df_bn.loc[df_bn['title_lang'] == 'fa']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang
19791,মুহাম্মদী বুক হাউজ,বাঁধাই হাদিস,No ISBN,No Ratings,No Reviews,390,No image,Muhammadi Book House,0.0,No summary,سنن ابى داود (আবু দাউদ শরীফ ),bn,No,bn,bn,বাঁধাই হাদিস,fa
19792,মাওলানা সাঈদ আলমিসবাহ,জামাত মেশকাত,No ISBN,1,No Review,300,No image,Muhammadi Book House,5.0,No summary,اصول بزدوي (উসূলে বায়দভী),bn,No,bn,bn,জামাত মেশকাত,fa


In [19]:
# df_bn.loc[df_bn['title_lang'] == 'ur']


In [12]:
df_bn.loc[df_bn['title_lang'] == 'ar']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang
8382,মাওলানা জাফর আহমদ উসমানী,আরবি ও উর্দু ভাষার বই,No ISBN,No Ratings,No Reviews,10000,No image,Darul Fikor Boirut,0.00,No summary,إعلاء السنن (ইলাউস সুনান (১-২১ খণ্ড),bn,No,bn,bn,"আরবি, উর্দু ভাষার বই",ar
8528,ইমাম মালেক বিন আনাছ রহ,বাঁধাই হাদিস,No ISBN,No Ratings,No Reviews,780,No image,Darul Hadith Cairo,0.00,No summary,الموطاء لمالك (আল মুয়াত্তা লি মালেক ),bn,No,bn,bn,বাঁধাই হাদিস,ar
8535,আল্লামা ইবনে কাছীর রহ,আরবি ও উর্দু বই,No ISBN,1,No Review,500,No image,Darul Hadith Cairo,5.00,No summary,قصص الأنبياء (কাসাসুল আম্বিয়া),bn,No,bn,bn,"আরবি, উর্দু বই",ar
8551,মাকতাবাতুল ফাতাহ বাংলাদেশ,আরবি ও উর্দু বই,No ISBN,No Ratings,No Reviews,800,No image,Darul Hadith Cairo,0.00,No summary,الإختيار لتعليل المختار (আল ইখতিয়ার লি তালিলিল...,bn,No,bn,bn,"আরবি, উর্দু বই",ar
8563,শায়খ ওয়ালীউদ্দিন মুহাম্মদ ইবনে আবদুল্লাহ,জামাত মেশকাত,No ISBN,No Ratings,No Reviews,1500,No image,Darul Hadith Cairo,0.00,No summary,মেশকাত শরীফ (১-৩ খণ্ড) مشكوة المصابيح,bn,No,bn,bn,জামাত মেশকাত,ar
8568,মাকতাবাতুল ফাতাহ বাংলাদেশ,আরবি ও উর্দু বই,No ISBN,1,No Review,800,No image,Darul Hadith Cairo,5.00,No summary,الموافقات في اصول الشريعة (আল মুয়াফাকাত ফী উসু...,bn,No,bn,bn,"আরবি, উর্দু বই",ar
8580,ফুয়াদ আবদুল বাকী,আরবি ও উর্দু ভাষার বই,No ISBN,2,No Review,900,No image,Darul Hadith Cairo,5.00,No summary,المعجم المفهرس لألفاظ القرآن الكريم (আল মুজামু...,bn,No,bn,bn,"আরবি, উর্দু ভাষার বই",ar
8583,আল্লামা জালালুদ্দীন সুয়ূতি রহ,আরবি ও উর্দু ভাষার বই,No ISBN,6,No Review,850,https://ds.rokomari.store/rokomari110/ProductN...,Darul Hadith Cairo,4.67,No summary,الإتقان في علوم القرآن (আল ইতকান ফী উলুমিল কুর...,bn,No,bn,bn,"আরবি, উর্দু ভাষার বই",ar
8585,হুজ্জাতুল ইসলাম ইমাম গাযযালী রহ,আরবি ও উর্দু বই,No ISBN,4,No Review,2800,No image,Darul Hadith Cairo,5.00,No summary,إحياء علوم الدين (ইহইয়াউ উলুমিদ্দীন (১-৫ খণ্ড),bn,No,bn,bn,"আরবি, উর্দু বই",ar
16982,ইমাম মুহাম্মাদ যাহিদআলকাওছারীরহ,আরবি ও উর্দু বই,No ISBN,No Ratings,No Reviews,138,https://ds.rokomari.store/rokomari110/ProductN...,Maktabatul Afnan,0.00,No summary,আলিম ওয়ালমুতায়াল্লিম(العالم والمتعلم),bn,No,bn,bn,"আরবি, উর্দু বই",ar


## 2.1 Trunicate other languages in mixed bangla titles

In [17]:
# title = "all quiet on the western front: পশ্চিম ফ্রন্টে সব শান্ত?"
# print(keep_only_bangla(title))

all quiet on the western front: পশ্চিম ফ্রন্টে সব শান্ত


In [13]:
df_bn["bangla_title"]= df_bn['title'].apply(keep_only_bangla)


In [14]:
df_bn

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
0,লু স্যুন,পশ্চিমবঙ্গের বই: সমকালীন গল্প,9788194097303,No Ratings,No Reviews,320,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,নির্বাচিত গল্প সংকলন,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সমকালীন গল্প",bn,নির্বাচিত গল্প সংকলন
1,আর্কাদি গাইদার,পশ্চিমবঙ্গের বই: শিশু-কিশোর উপন্যাস,9788193860939,No Ratings,No Reviews,180,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,তিমুর ও তার দলবল,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, শিশু-কিশোর উপন্যাস",bn,তিমুর ও তার দলবল
2,মতি নন্দী,পশ্চিমবঙ্গের বই: সমকালীন উপন্যাস,No ISBN,No Ratings,No Reviews,250,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,নায়কের প্রবেশ ও প্রস্থান,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, সমকালীন উপন্যাস",bn,নায়কের প্রবেশ ও প্রস্থান
3,উৎপলকুমার বসু,পশ্চিমবঙ্গের বই: বাংলা কবিতা,No ISBN,No Ratings,No Reviews,300,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,হাঁস চলার পথ,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, বাংলা কবিতা",bn,হাঁস চলার পথ
4,দিলরুবা হাসান,বয়স যখন ৪-৮: শিক্ষামূলক,No ISBN,No Ratings,No Reviews,129,No image,বৃত্তকলা একাডেমি,0.0,No summary,অনুভূতির ছোঁয়া,bn,No,bn,bn,"বয়স যখন ৪-৮, শিক্ষামূলক",bn,অনুভূতির ছোঁয়া
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98249,আ শ ম বাবর আলী,বয়স যখন ৪-৮: বাংলা ছড়া,9789849338253,No Ratings,No Reviews,129,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থ প্রকাশ,0.0,No summary,আবোল তাবোল ছড়া,bn,No,bn,bn,"বয়স যখন ৪-৮, বাংলা ছড়া",bn,আবোল তাবোল ছড়া
98250,বিলাল হোসাইন নূরী,বয়স যখন ৮-১২: গল্প,9789849418405,No Ratings,No Reviews,144,https://ds.rokomari.store/rokomari110/ProductN...,গ্রন্থকুঞ্জ প্রকাশন,0.0,No summary,কুসুম ফোটার বেলা,bn,No,bn,bn,"বয়স যখন ৮-১২, গল্প",bn,কুসুম ফোটার বেলা
98251,সতীনাথ ভাদুড়ী,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,No ISBN,No Ratings,No Reviews,360,No image,গ্রন্থালয় (ভারত),0.0,No summary,সতীনাথ ভাদুড়ীঃ শতবার্ষিকী রচনা সংকলন ২য় খণ্ড,bn,No,bn,bn,"পশ্চিমবঙ্গের বই, রচনাসমগ্র, সংকলন",bn,সতীনাথ ভাদুড়ীঃ শতবার্ষিকী রচনা সংকলন ২য় খণ্ড
98252,চিত্তরঞ্জন ঘোষাল,হিন্দু ধর্মীয় বই,No ISBN,No Ratings,No Reviews,720,No image,গ্রন্থিক (ভারত),0.0,No summary,গীতা সংগ্রহ: শ্রীমদভগবদ্গীতা-সহ ৩৫টি গীতা একখণ...,bn,No,bn,bn,হিন্দু ধর্মীয় বই,bn,গীতা সংগ্রহ: শ্রীমদভগবদ্গীতাসহ ৩৫টি গীতা একখণ্...


In [15]:
df_bn.loc[df_bn['title_lang'] == 'ar']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
8382,মাওলানা জাফর আহমদ উসমানী,আরবি ও উর্দু ভাষার বই,No ISBN,No Ratings,No Reviews,10000,No image,Darul Fikor Boirut,0.00,No summary,إعلاء السنن (ইলাউস সুনান (১-২১ খণ্ড),bn,No,bn,bn,"আরবি, উর্দু ভাষার বই",ar,ইলাউস সুনান ১২১ খণ্ড
8528,ইমাম মালেক বিন আনাছ রহ,বাঁধাই হাদিস,No ISBN,No Ratings,No Reviews,780,No image,Darul Hadith Cairo,0.00,No summary,الموطاء لمالك (আল মুয়াত্তা লি মালেক ),bn,No,bn,bn,বাঁধাই হাদিস,ar,আল মুয়াত্তা লি মালেক
8535,আল্লামা ইবনে কাছীর রহ,আরবি ও উর্দু বই,No ISBN,1,No Review,500,No image,Darul Hadith Cairo,5.00,No summary,قصص الأنبياء (কাসাসুল আম্বিয়া),bn,No,bn,bn,"আরবি, উর্দু বই",ar,কাসাসুল আম্বিয়া
8551,মাকতাবাতুল ফাতাহ বাংলাদেশ,আরবি ও উর্দু বই,No ISBN,No Ratings,No Reviews,800,No image,Darul Hadith Cairo,0.00,No summary,الإختيار لتعليل المختار (আল ইখতিয়ার লি তালিলিল...,bn,No,bn,bn,"আরবি, উর্দু বই",ar,আল ইখতিয়ার লি তালিলিল মুখতার ১২ খণ্ড
8563,শায়খ ওয়ালীউদ্দিন মুহাম্মদ ইবনে আবদুল্লাহ,জামাত মেশকাত,No ISBN,No Ratings,No Reviews,1500,No image,Darul Hadith Cairo,0.00,No summary,মেশকাত শরীফ (১-৩ খণ্ড) مشكوة المصابيح,bn,No,bn,bn,জামাত মেশকাত,ar,মেশকাত শরীফ ১৩ খণ্ড
8568,মাকতাবাতুল ফাতাহ বাংলাদেশ,আরবি ও উর্দু বই,No ISBN,1,No Review,800,No image,Darul Hadith Cairo,5.00,No summary,الموافقات في اصول الشريعة (আল মুয়াফাকাত ফী উসু...,bn,No,bn,bn,"আরবি, উর্দু বই",ar,আল মুয়াফাকাত ফী উসুলিশ শরীয়াহ ১৪ খণ্ড
8580,ফুয়াদ আবদুল বাকী,আরবি ও উর্দু ভাষার বই,No ISBN,2,No Review,900,No image,Darul Hadith Cairo,5.00,No summary,المعجم المفهرس لألفاظ القرآن الكريم (আল মুজামু...,bn,No,bn,bn,"আরবি, উর্দু ভাষার বই",ar,আল মুজামুল মুফাহরাস লি আলফাযিল কুরআন
8583,আল্লামা জালালুদ্দীন সুয়ূতি রহ,আরবি ও উর্দু ভাষার বই,No ISBN,6,No Review,850,https://ds.rokomari.store/rokomari110/ProductN...,Darul Hadith Cairo,4.67,No summary,الإتقان في علوم القرآن (আল ইতকান ফী উলুমিল কুর...,bn,No,bn,bn,"আরবি, উর্দু ভাষার বই",ar,আল ইতকান ফী উলুমিল কুরআন ১২ খণ্ড
8585,হুজ্জাতুল ইসলাম ইমাম গাযযালী রহ,আরবি ও উর্দু বই,No ISBN,4,No Review,2800,No image,Darul Hadith Cairo,5.00,No summary,إحياء علوم الدين (ইহইয়াউ উলুমিদ্দীন (১-৫ খণ্ড),bn,No,bn,bn,"আরবি, উর্দু বই",ar,ইহইয়াউ উলুমিদ্দীন ১৫ খণ্ড
16982,ইমাম মুহাম্মাদ যাহিদআলকাওছারীরহ,আরবি ও উর্দু বই,No ISBN,No Ratings,No Reviews,138,https://ds.rokomari.store/rokomari110/ProductN...,Maktabatul Afnan,0.00,No summary,আলিম ওয়ালমুতায়াল্লিম(العالم والمتعلم),bn,No,bn,bn,"আরবি, উর্দু বই",ar,আলিম ওয়ালমুতায়াল্লিম


In [29]:
df_bn.loc[df_bn['title_lang'] == 'ur']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
63148,সাইয়েদ আহমদ মিয়াল,জামাত মিজান,No ISBN,No Ratings,No Reviews,11,No image,Maktabatul Fatah Bangladesh,0.0,No summary,তারীখুল ইসলাম ১ম খণ্ড (تاريخ الاسلام حصئ اول ),bn,No,bn,bn,জামাত মিজান,ur,তারীখুল ইসলাম ১ম খণ্ড


In [16]:
df_bn.loc[df_bn['title_lang'] == 'fa']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
19791,মুহাম্মদী বুক হাউজ,বাঁধাই হাদিস,No ISBN,No Ratings,No Reviews,390,No image,Muhammadi Book House,0.0,No summary,سنن ابى داود (আবু দাউদ শরীফ ),bn,No,bn,bn,বাঁধাই হাদিস,fa,আবু দাউদ শরীফ
19792,মাওলানা সাঈদ আলমিসবাহ,জামাত মেশকাত,No ISBN,1,No Review,300,No image,Muhammadi Book House,5.0,No summary,اصول بزدوي (উসূলে বায়দভী),bn,No,bn,bn,জামাত মেশকাত,fa,উসূলে বায়দভী


In [17]:
df_bn['bangla_title'].apply(detect_language).value_counts()


,count
bangla_title,
bn,98254


# 3. Look for duplicated titles

In [20]:
# df_bn[df_bn.duplicated(subset=['title'], keep=False)]
pd.concat(g for _, g in df_bn.groupby("bangla_title") if len(g) > 1)

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
9953,এসেনশিয়াল পাবলিকেশন সম্পাদক,টোফেল,No ISBN,7,7 Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,4.86,No summary,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,No,bn,bn,টোফেল,bn,Cliffs Toefl Grand Review বাংলা ভার্সন
9954,এসেনশিয়াল পাবলিকেশন সম্পাদক,টোফেল,No ISBN,No Ratings,No Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,0.00,CONTENTS,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,Yes,bn,bn,টোফেল,bn,Cliffs Toefl Grand Review বাংলা ভার্সন
4650,মাইদুল ইসলাম প্রধান মুকুল,অন্যান্য চাকরির প্রস্তুতি,No ISBN,3,2 Reviews,480,https://ds.rokomari.store/rokomari110/ProductN...,বিদ্যাবাড়ি পাবলিকেশন্স,4.50,“গত বছর ১৮তম শিক্ষক নিবন্ধন পরীক্ষায় সর্বোচ্চ ...,NTRCA ডাইজেস্ট প্লাস,bn,Yes,bn,bn,অন্যান্য চাকরির প্রস্তুতি,bn,NTRCA ডাইজেস্ট প্লাস
4867,মাইদুল ইসলাম প্রধান মুকুল,অন্যান্য চাকরির প্রস্তুতি,No ISBN,3,1 Review,420,No image,বিদ্যাবাড়ি পাবলিকেশন্স,4.33,বিদ্যাবাড়ি নিবন্ধন ডাইজেস্ট প্লাস বইটির বৈশিষ...,NTRCA ডাইজেস্ট প্লাস,bn,Yes,bn,bn,অন্যান্য চাকরির প্রস্তুতি,bn,NTRCA ডাইজেস্ট প্লাস
44609,হুমায়ূন আহমেদ,সায়েন্স ফিকশন গল্প,No ISBN,14,4 Reviews,198,https://ds.rokomari.store/rokomari110/ProductN...,সাগর পাবলিশার্স,5.00,No summary,অঁহক,bn,No,bn,bn,সায়েন্স ফিকশন গল্প,bn,অঁহক
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,আলহাজ্জ মাওলানা এ কে এম ফজলুর রহমান মুন্সী,রেসিপি,No ISBN,No Ratings,No Reviews,105,No image,বাংলাদেশ তাজ কোম্পানী লিঃ,0.00,No summary,৫০০ রকমের রান্না,bn,No,bn,bn,রেসিপি,bn,৫০০ রকমের রান্না
45218,জুল ভার্ন,শিশু-কিশোর উপন্যাস,9789849129080,3,No Review,118,https://ds.rokomari.store/rokomari110/ProductN...,সবুজপাতা,4.33,No summary,৮০ দিনে বিশ্বভ্রমণ,bn,No,bn,bn,শিশু-কিশোর উপন্যাস,bn,৮০ দিনে বিশ্বভ্রমণ
49464,জুল ভার্ন,"বয়স যখন ১২-১৭: রহস্য, গোয়েন্দা, থ্রিলার ও অ্য...",9789849310587,4,4 Reviews,99,https://ds.rokomari.store/rokomari110/ProductN...,প্রচলন প্রকাশন,5.00,No summary,৮০ দিনে বিশ্বভ্রমণ,bn,No,bn,bn,"বয়স যখন ১২-১৭, রহস্য, গোয়েন্দা, থ্রিলার, অ্যা...",bn,৮০ দিনে বিশ্বভ্রমণ
27581,হেলেন হ্যানফ,অনুবাদ উপন্যাস,No ISBN,3,1 Review,216,https://ds.rokomari.store/rokomari110/ProductN...,শিরোনাম প্রকাশন,4.67,আজ থেকে প্রায় সত্তর বছর আগে তিন হাজার মাইল দূর...,৮৪ চ্যারিং ক্রস রোড,bn,Yes,bn,bn,অনুবাদ উপন্যাস,bn,৮৪ চ্যারিং ক্রস রোড


In [46]:
df_bn.loc[df_bn['bangla_title'] == '৮০ দিনে বিশ্বভ্রমণ']


,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
45218,জুল ভার্ন,শিশু-কিশোর উপন্যাস,9789849129080,3,No Review,118,https://ds.rokomari.store/rokomari110/ProductN...,সবুজপাতা,4.33,No summary,৮০ দিনে বিশ্বভ্রমণ,bn,No,bn,bn,শিশু-কিশোর উপন্যাস,bn,৮০ দিনে বিশ্বভ্রমণ
49464,জুল ভার্ন,"বয়স যখন ১২-১৭: রহস্য, গোয়েন্দা, থ্রিলার ও অ্য...",9789849310587,4,4 Reviews,99,https://ds.rokomari.store/rokomari110/ProductN...,প্রচলন প্রকাশন,5.00,No summary,৮০ দিনে বিশ্বভ্রমণ,bn,No,bn,bn,"বয়স যখন ১২-১৭, রহস্য, গোয়েন্দা, থ্রিলার, অ্যা...",bn,৮০ দিনে বিশ্বভ্রমণ


In [22]:
# def same_merge(x):
#     return ','.join(x[x.notnull()].astype(str))


In [42]:
# duplicated_df = df_bn[df_bn.duplicated(subset=['bangla_title'], keep=False)]
duplicated_df = df_bn[df_bn.duplicated(subset=['bangla_title', 'author', 'categories_fixed'], keep=False)]
pd.concat(g for _, g in duplicated_df.groupby("bangla_title") if len(g) > 1)

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories,categories_fixed,title_lang,bangla_title
9953,এসেনশিয়াল পাবলিকেশন সম্পাদক,টোফেল,No ISBN,7,7 Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,4.86,No summary,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,No,bn,bn,টোফেল,bn,Cliffs Toefl Grand Review বাংলা ভার্সন
9954,এসেনশিয়াল পাবলিকেশন সম্পাদক,টোফেল,No ISBN,No Ratings,No Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,0.00,CONTENTS,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,Yes,bn,bn,টোফেল,bn,Cliffs Toefl Grand Review বাংলা ভার্সন
4650,মাইদুল ইসলাম প্রধান মুকুল,অন্যান্য চাকরির প্রস্তুতি,No ISBN,3,2 Reviews,480,https://ds.rokomari.store/rokomari110/ProductN...,বিদ্যাবাড়ি পাবলিকেশন্স,4.50,“গত বছর ১৮তম শিক্ষক নিবন্ধন পরীক্ষায় সর্বোচ্চ ...,NTRCA ডাইজেস্ট প্লাস,bn,Yes,bn,bn,অন্যান্য চাকরির প্রস্তুতি,bn,NTRCA ডাইজেস্ট প্লাস
4867,মাইদুল ইসলাম প্রধান মুকুল,অন্যান্য চাকরির প্রস্তুতি,No ISBN,3,1 Review,420,No image,বিদ্যাবাড়ি পাবলিকেশন্স,4.33,বিদ্যাবাড়ি নিবন্ধন ডাইজেস্ট প্লাস বইটির বৈশিষ...,NTRCA ডাইজেস্ট প্লাস,bn,Yes,bn,bn,অন্যান্য চাকরির প্রস্তুতি,bn,NTRCA ডাইজেস্ট প্লাস
79664,ইয়া পেরেলমান,গণিত,No ISBN,No Ratings,No Reviews,90,No image,বর্তমান সময়,0.00,No summary,অঙ্কের মজা,bn,No,bn,bn,গণিত,bn,অঙ্কের মজা
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31819,পারভেজ ইসলাম,বাংলা কবিতা,No ISBN,No Ratings,No Reviews,275,https://ds.rokomari.store/rokomari110/ProductN...,ওয়ার্থী পাবলিকেশনস,0.00,"১৯ কবিতার কুড়ি বছর, বইটি হয়তো আপনাকে প্রকৃতি...",১৯ কবিতার কুড়ি বছর,bn,Yes,bn,bn,বাংলা কবিতা,bn,১৯ কবিতার কুড়ি বছর
19534,মাওলানা এম আর বারী সম্পাদক,আরবি শিক্ষা,No ISBN,No Ratings,No Reviews,84,https://ds.rokomari.store/rokomari110/ProductN...,মনিহার বুক ডিপো,0.00,No summary,৩০ দিনে আধুনিক পদ্ধতিতে আরবী ভাষা শিক্ষা,bn,No,bn,bn,আরবি শিক্ষা,bn,৩০ দিনে আধুনিক পদ্ধতিতে আরবী ভাষা শিক্ষা
62452,মাওলানা এম আর বারী সম্পাদক,আরবি শিক্ষা,No ISBN,2,1 Review,126,https://ds.rokomari.store/rokomari110/ProductN...,Maliha Prokashoni,4.50,No summary,৩০ দিনে আধুনিক পদ্ধতিতে আরবী ভাষা শিক্ষা,bn,No,bn,bn,আরবি শিক্ষা,bn,৩০ দিনে আধুনিক পদ্ধতিতে আরবী ভাষা শিক্ষা
5620,তানভীর মেহেদী,সমকালীন উপন্যাস,No ISBN,No Ratings,No Reviews,215,https://ds.rokomari.store/rokomari110/ProductN...,বইবাজার প্রকাশনী,0.00,সস্তা পারফিউমের গন্ধ ছড়িয়ে শিউলি আমার ঘরে এল। ...,৩৬/ডি,bn,Yes,bn,bn,সমকালীন উপন্যাস,bn,৩৬ডি
